In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
from supabase import create_client, Client
import dotenv
dotenv.load_dotenv()
import os
url: str = os.environ.get("URL")
key: str = os.environ.get("ANON_KEY")
supabase: Client = create_client(url, "sb_secret__hURMlWV0e8lHlKreudFAA_UBScLp6U")

In [3]:
response = requests.get('https://s3.amazonaws.com/tripdata/index')

In [4]:
response = requests.get("https://s3.amazonaws.com/tripdata")

In [5]:
response.raise_for_status()  # Ensure we notice bad responses
soup = BeautifulSoup(response.text, 'xml')

In [6]:
soup.find_all('a')

[]

In [14]:
current_files = parse_citibike_xml_to_df(response.text)

NameError: name 'parse_citibike_xml_to_df' is not defined

In [8]:
import json 

In [9]:
json_str = current_files.head(10).to_json(orient="records", date_format='iso')
files_data = json.loads(json_str)
supabase.table('processed_files').upsert(files_data, on_conflict='file_name').execute()

NameError: name 'current_files' is not defined

In [10]:
result = supabase.table("processed_files").select("*").execute()

In [11]:
prev_files = pd.DataFrame(result.data)

In [12]:
import pandas as pd


def get_new_files(live_files, prev_files):
    # Method 1: Using merge with indicator
    merged = live_files.merge(
        prev_files,
        on="file_name",
        how="left",
        suffixes=("_new", "_old"),
        indicator=True,
    )

    # Filter for rows that either:
    # 1) Don't appear in old_df (_merge == 'left_only')
    # 2) Appear in old_df but have newer last_modified date
    filtered_df = merged[
        (merged["_merge"] == "left_only")  # Not in old_df
        | (
            merged["last_modified_new"] > merged["last_modified_old"]
        )  # Newer modification date
    ][["file_name", "last_modified_new"]].rename(
        columns={"last_modified_new": "last_modified"}
    )
    return filtered_df

In [13]:
filtered_df

NameError: name 'filtered_df' is not defined

In [20]:
def get_prev_processed_files():
    result = supabase.table("processed_files").select("*").execute()
    prev_files = pd.DataFrame(result.data)
    return prev_files

In [21]:
def get_live_files():
    response = requests.get("https://s3.amazonaws.com/tripdata")
    response.raise_for_status()  # Ensure we notice bad responses
    current_files = parse_citibike_xml_to_df(response.text)
    return current_files

In [22]:
def get_unprocessed_files():
    prev_files = get_prev_processed_files()
    # Files live on the website.
    live_files = get_live_files()
    # Get new files which have not been processed.
    new_files = get_new_files(live_files, prev_files)
    new_files["locale"] = new_files["file_name"].apply(
        lambda x: "JC" if x.startswith("JC") else "NYC"
    )
    return new_files

In [23]:
new_files = get_unprocessed_files()
new_files.to_csv("new_files_test.csv", index=False)

In [15]:
def parse_citibike_xml_to_df(xml_content):
    """
    Parse the Citibike S3 bucket XML listing into a pandas DataFrame

    Args:
        xml_content (str): The XML content from the S3 bucket listing

    Returns:
        pd.DataFrame: DataFrame with columns 'filename', 'last_modified', 'size_bytes'
    """

    # Parse the XML
    soup = BeautifulSoup(xml_content, "xml")

    # Find all Contents elements
    contents = soup.find_all("Contents")

    # Extract data from each file
    file_data = []

    for content in contents:
        # Get the filename (Key)
        key = content.find("Key")
        filename = key.text if key else None

        # Get the last modified date
        last_modified = content.find("LastModified")
        last_modified_date = last_modified.text if last_modified else None

        # Get the file size
        size = content.find("Size")
        size_bytes = int(size.text) if size else None

        # Add to our data list
        if filename:  # Only add if we have a filename
            file_data.append(
                {
                    "file_name": filename,
                    "last_modified": last_modified_date,
                }
            )

    # Create DataFrame
    df = pd.DataFrame(file_data)

    # Convert last_modified to datetime
    if not df.empty and "last_modified" in df.columns:
        df["last_modified"] = pd.to_datetime(df["last_modified"])

    return df